# Pytorch多分类实战（数据为sklearn的红酒）

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import numpy as np
#Variable类已经被禁用，现在tensor能够追踪
#from torch.autograd import Variable
torch.__version__

'1.5.0'

# softmax分类实例
## 数据集
由sklean加载的wine数据集  
*训练集：133个样本，3个类别  
*测试集：45个样本，3个类别

In [55]:
#加载数据
wine_train = np.load("./data_wine.npy")
wine_test = np.load("./data_wine_test.npy")
#归一化
n,l=wine_train.shape
for j in range(l-1):
    meanVal=np.mean(wine_train[:,j])
    stdVal=np.std(wine_train[:,j])
    wine_train[:,j]=(wine_train[:,j]-meanVal)/stdVal
#将训练数据和训练标签分开
wine_traindata = wine_train[:,0:13]
wine_trainlabel = wine_train[:,-1]
#将numpy转化为tensor张量
wine_traindata = torch.from_numpy(wine_traindata)
wine_trainlabel = torch.from_numpy(wine_trainlabel)
#梯度只能计算浮点张量，所以类型都要转化为float 
wine_traindata = wine_traindata.float()
wine_trainlabel = wine_trainlabel.long()
#对测试集采取相同操作方式
n,l=wine_test.shape
for j in range(l-1):
    meanVal=np.mean(wine_test[:,j])
    stdVal=np.std(wine_test[:,j])
    wine_test[:,j]=(wine_test[:,j]-meanVal)/stdVal
wine_testdata = wine_test[:,0:13]
wine_testlabel = wine_test[:,-1]

wine_testdata = torch.from_numpy(wine_testdata)
wine_testlabel = torch.from_numpy(wine_testlabel)
    
wine_testdata = wine_testdata.float()
wine_testlabel = wine_testlabel.long()

## 神经网络
构建两层神经网络，只有一层隐藏层，激活函数采用relu

In [56]:
#构建神经网络
class Net(nn.Module):#继承父类
    def __init__(self):
        super(Net, self).__init__()
        #构建feed forward网络
        self.fc1 = nn.Linear(13, 7)
        self.fc2 = nn.Linear(7, 3)
        
    def forward(self, x):
        #x = self.fc1(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        out = F.softmax(x, dim=1)
        return out
    
net = Net()
print(net)

optimizer = torch.optim.SGD(net.parameters(),lr=0.02) #设置优化器参数,lr=0.002指的是学习率的大小
criterion=nn.CrossEntropyLoss()

Net(
  (fc1): Linear(in_features=13, out_features=7, bias=True)
  (fc2): Linear(in_features=7, out_features=3, bias=True)
)


总共训练1000次，这里没有分batch，一次训练使用全部数据

In [57]:
#train
epochs = 1000
for i in range(epochs):
    out = net(wine_traindata)
    #loss
    loss=criterion(out,wine_trainlabel)
    optimizer.zero_grad()
    loss.backward()
    #update
    optimizer.step()
    if (i+1)%100 ==0 : # 这里我们每100次输出相关的信息
        # 指定模型为计算模式
        net.eval()
        print("Epoch:{},Loss:{:.4f}".format(i+1,loss.item()))

Epoch:100,Loss:0.9888
Epoch:200,Loss:0.9057
Epoch:300,Loss:0.8463
Epoch:400,Loss:0.8060
Epoch:500,Loss:0.7752
Epoch:600,Loss:0.7477
Epoch:700,Loss:0.7223
Epoch:800,Loss:0.6988
Epoch:900,Loss:0.6771
Epoch:1000,Loss:0.6569


使用测试集进行测试，得到准确率

In [58]:
test_out = net(wine_testdata)

def test(pred,lab):
    t=pred.max(-1)[1]==lab
    return torch.mean(t.float())

acc =  test(test_out, wine_testlabel)
print(acc)

tensor(0.9111)
